# Train an IntegratedML model on Readmission Dataset
## Use JDBC to connect to InterSystems IRIS database
This Notebook demonstrates:
- Using the JayDeBeApi Python library to connect to InterSystems IRIS
- Creating views to segment data into training and test sets
- Defining and training an IntegratedML model to predict marketing campaign responses
- Comparing the resulting model's predictions to data in the test set (that the model was not trained on)
- Using the IntegratedML "VALIDATE MODEL" command to calculate accuracy metrics on the test set data

In [ ]:
!conda activate
!pip3 install jupyterthemes

In [ ]:
!jt -r

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

!ls /tf/intersystems-jdbc-3.1.0.jar
RUN apt-get update && \
    DEBIAN_FRONTEND=noninteractive \
    apt-get -y install default-jre-headless && \
    apt-get clean && \
    rm -rf /var/lib/apt/lists/*

In [ ]:
!pip install JayDeBeApi

In [ ]:
!pip list JPype1

### 1. Set environment variables, if necessary

In [ ]:
#import os
#os.environ['JAVA_HOME']='C:\Progra~1\Java\jdk1.8.0_241'
#os.environ['CLASSPATH'] = 'C:\interSystems\IRIS20194\dev\java\lib\JDK18\intersystems-jdbc-3.0.0.jar'
#os.environ['HADOOP_HOME']='C:\hadoop\bin'  #winutil binary must be in Hadoop's Home

### 2. Get jdbc connection and cursor

In [ ]:

import jaydebeapi
url = "jdbc:IRIS://irisimlsvr:1972/USER" #"jdbc:IRIS://172.17.0.1:8091/USER"
driver = 'com.intersystems.jdbc.IRISDriver'
user = "SUPERUSER"
password = "SYS"
#libx = "C:/InterSystems/IRIS20194/dev/java/lib/JDK18"
#jarfile = "C:/InterSystems/IRIS20194/dev/java/lib/JDK18/intersystems-jdbc-3.0.0.jar"
jarfile = "./intersystems-jdbc-3.1.0.jar"

In [ ]:
conn = jaydebeapi.connect(driver, url, [user, password], jarfile)
curs = conn.cursor()

### 3. specify the source data table

In [ ]:
dataTable = 'Patient.Readmission'

### 4. Execute a query and display results in Pandas DataFrame

In [ ]:
import pandas as pd
from IPython.display import display

df = pd.read_sql("select TOP 20 * from %s" % dataTable, conn)
display(df)

In [ ]:
# Show number rows
df1 = pd.read_sql("SELECT COUNT(*) FROM %s" % dataTable, conn)
display(df1)

### Cleaning before retrying

In [ ]:
curs.execute("DROP VIEW Patient.ReadmissionTraining")
curs.execute("DROP VIEW Patient.ReadmissionPredict")
curs.execute("DROP MODEL ReadmissionModel")

### 5. Make some views to split training and testing datasets

In [ ]:
df1 = pd.read_sql("SELECT COUNT(*) FROM %s WHERE  ID>=11000" % dataTable, conn)
display(df1)

In [ ]:
# Training set view
curs.execute("CREATE VIEW Patient.ReadmissionTraining AS SELECT * FROM Patient.Readmission WHERE ID<11000")
# Prediction set
curs.execute("CREATE VIEW Patient.ReadmissionPredict AS SELECT * FROM Patient.Readmission WHERE ID>=11000")

### 6. Look at Data

In [ ]:
display(pd.read_sql("select TOP 20 * from Patient.ReadmissionTraining", conn))

In [ ]:
display(pd.read_sql("SELECT COUNT(*) rowCount FROM Patient.ReadmissionTraining", conn))

In [ ]:
df.dtypes

In [ ]:
len(df.dtypes)

### 7. Choose ML framework with "SET ML CONFIGURATION"


In [ ]:
curs.execute("SET ML CONFIGURATION %AutoML")

### 8. Create and Train an IntegratedML Model using default settings
IntegratedML only needs a model name, the name of the column that is the target column to predict, and a table (or SELECT query to specify input columns.

This is a simple DDL query that executes immediately.

In [ ]:
res = curs.execute("CREATE MODEL ReadmissionModel PREDICTING (MxWillReAdmit) FROM Patient.ReadmissionTraining")

Now that the model is defined, you can TRAIN it, which invokes the AutoML machine learning procedure.

In [ ]:
curs.execute("TRAIN MODEL ReadmissionModel")

Once that finishes, you can see some information about the model in the "ML_TRAINED_MODELS" table.

In [ ]:
df3 = pd.read_sql("SELECT * FROM INFORMATION_SCHEMA.ML_TRAINED_MODELS", conn)
display(df3)

### 9. Compare model output to data it has not seen yet
Now you can use SQL to SELECT data from another table, run the IntegratedML model on this new data, and see how well the predictions match the data!

In [ ]:
df4 = pd.read_sql("SELECT PREDICT(ReadmissionModel) AS PredictedReadmit, \
                  MxWillReAdmit AS ActualReadmit FROM Patient.ReadmissionPredict", conn)
display(df4)

### 10. VALIDATE MODEL command calculates accuracy metrics
You can certainly take that output above and calculate the accuracy using a standard formula, but IntegratedML has a built-in function to do that!

Each time you run the command "VALIDATE MODEL..." it generates a set of metrics calculated on the data passed into the query. Since this table can be a bit difficult to read in its raw form we use a simple "pivot" call to arrange the data.

In [ ]:
curs.execute("VALIDATE MODEL ReadmissionModel FROM Patient.ReadmissionPredict")
df5 = pd.read_sql("SELECT * FROM INFORMATION_SCHEMA.ML_VALIDATION_METRICS", conn)
df6 = df5.pivot(index='VALIDATION_RUN_NAME', columns='METRIC_NAME', values='METRIC_VALUE')
display(df6)

### 9. Query that highlights incorrectly predicted rows

This query retrieves encounters the model incorrectly scored, along with the probability estimate of the prediction.

In [ ]:
display(pd.read_sql('''
SELECT
	PROBABILITY(ReadmissionModel FOR '1') ReadmissionProbability,
	PREDICT(ReadmissionModel) PredictedReadmission,
	MxWillReAdmit ActualReadmission,
	*
FROM
	Patient.ReadmissionPredict
WHERE
	MxWillReAdmit = 1
	AND MxWillReAdmit != PREDICT(ReadmissionModel)
''', conn))

### 9. Query that pulls recent Encounters and filters by those with high probability of readmission

In [ ]:
display(pd.read_sql('''
SELECT 
	PROBABILITY(ReadmissionModel FOR '1') ReadmissionProbability,
	PREDICT(ReadmissionModel) PredictedReadmission,
	*
FROM
	Patient.ReadmissionPredict
WHERE
	MxEncounterEndYear >=2019 AND
	MxEndDateMonth >= 12 AND
    PROBABILITY(ReadmissionModel FOR '1') >= 0.25
    ''', conn))